In [1]:
from transformers import AutoModelForMaskedLM

model_checkpoint = "nlp-magnets/magbert"
model = AutoModelForMaskedLM.from_pretrained(model_checkpoint)

In [2]:
text = "Magnetite is the oxide of [MASK]."

In [3]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

In [4]:
import torch

inputs = tokenizer(text, return_tensors="pt")
token_logits = model(**inputs).logits
# Find the location of [MASK] and extract its logits
mask_token_index = torch.where(inputs["input_ids"] == tokenizer.mask_token_id)[1]
mask_token_logits = token_logits[0, mask_token_index, :]
# Pick the [MASK] candidates with the highest logits


In [5]:
mask_token_logits

tensor([[-10.1050, -10.1693, -10.1650,  ..., -10.0200, -10.3236,  -7.6567]],
       grad_fn=<IndexBackward0>)

In [6]:
top_5_tokens = torch.topk(mask_token_logits, 5, dim=1).indices[0].tolist()

for token in top_5_tokens:
    print(f"'>>> {text.replace(tokenizer.mask_token, tokenizer.decode([token]))}'")

'>>> Magnetite is the oxide of iron.'
'>>> Magnetite is the oxide of oxygen.'
'>>> Magnetite is the oxide of titanium.'
'>>> Magnetite is the oxide of fe.'
'>>> Magnetite is the oxide of mn.'


In [25]:
torch.topk(mask_token_logits, 5, dim=1).values[0]

tensor([9.7153, 8.9145, 8.0896, 7.9685, 7.7463], grad_fn=<SelectBackward0>)

In [28]:
mask_token_logits[0].detach().numpy()

array([-10.105022 , -10.169344 , -10.164963 , ..., -10.019992 ,
       -10.323558 ,  -7.6566772], dtype=float32)

In [29]:
import numpy as np

def softmax(x):
    """Compute softmax values for each sets of scores in x."""
    e_x = np.exp(x - np.max(x))
    return e_x / e_x.sum()

scores = mask_token_logits[0].detach().numpy()
print(softmax(scores))

[7.1564060e-10 6.7105738e-10 6.7400507e-10 ... 7.7915313e-10 5.7515620e-10
 8.2793790e-09]


In [32]:
softmax(scores)[scores.argmax()]

0.290113